In [51]:
import json, cv2, numpy as np

In [52]:
ONNX_PATH   = ".\\artifacts\\crnn_nc92_k2x1.onnx"  # <- adjust to your file
CHARSET_JSON= ".\\artifacts\\charset.json"
IMG_H, IMG_W = 32, 128
BLANK_IDX = 0

# load charset

In [53]:
with open(CHARSET_JSON, "r") as f:
    CHARS = json.load(f)["chars"]
idx2char = {i+1: c for i, c in enumerate(CHARS)}

# CTC greedy decoder

In [54]:
def ctc_decode_from_logits(logits_np):
    if logits_np.ndim == 3:       # T x B x C
        argm = logits_np.argmax(2)[:, 0]
    elif logits_np.ndim == 2:     # T x C
        argm = logits_np.argmax(1)
    else:                         # 1 x T x C
        argm = logits_np[0].argmax(1)

    out, prev = [], BLANK_IDX
    for s in argm.tolist():
        if s != prev and s != BLANK_IDX:
            out.append(idx2char.get(int(s), ""))
        prev = s
    return "".join(out)

# preprocess: grayscale, keep aspect ratio, pad to 128x32, normalize to [0,1]

In [55]:
def preprocess_for_crnn(bgr, target_h=IMG_H, target_w=IMG_W):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    h, w = gray.shape
    scale = min(target_w / w, target_h / h)  # fit in target
    nw, nh = max(1, int(w * scale)), max(1, int(h * scale))

    resized = cv2.resize(gray, (nw, nh), interpolation=cv2.INTER_LINEAR)

    # create white canvas and paste resized centered (pad right/bottom)
    canvas = np.full((target_h, target_w), 255, np.uint8)
    y0 = (target_h - nh) // 2
    x0 = 0  # left-align in width direction (common in OCR)
    canvas[y0:y0+nh, x0:x0+nw] = resized

    # to float32 blob 1x1xH xW
    blob = canvas.astype(np.float32) / 255.0
    blob = blob[np.newaxis, np.newaxis, :, :]
    return blob

# load ONNX with OpenCV and run

In [56]:
net = cv2.dnn.readNet(ONNX_PATH)

def recognize_image_path(image_path):
    bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if bgr is None:
        raise FileNotFoundError(f"Could not read image: {image_path}")
    inp = preprocess_for_crnn(bgr)
    net.setInput(inp)
    logits = net.forward()
    return ctc_decode_from_logits(logits)

In [57]:
text = recognize_image_path(".\\artifacts\\000023.jpg")
print("Pred:", text)

Pred: Bloodgroudabs
